Planner + Researcher + Critic using LangChain + GROQ

MULTI-AGENT TRAVEL ASSISTANT (FLIGHT BOOKING)
🧠 What this agent will do

User says:

“Book the cheapest flight from Chennai to Delhi tomorrow.”

The system will:

🧠 Planner Agent → decides steps

🔍 Researcher Agent → decides what info is needed

⚙️ Executor Agent → fetches flights, selects cheapest, books

🧐 Critic Agent → verifies booking details

✅ Includes real-world actions (API simulation + file saving)

| Agent         | Role                                       |
| ------------- | ------------------------------------------ |
| 🧠 Planner    | Break booking task into steps              |
| 🔍 Researcher | Decide data needed (routes, dates, prices) |
| ⚙️ Executor   | Call flight APIs & book                    |
| 🧐 Critic     | Verify booking correctness                 |



In [1]:
!pip install langchain langchain-groq sentence-transformers faiss-cpu


In [16]:
import os
os.environ["GROQ_API_KEY"] = "key"

In [17]:
#import required modules

from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage


In [18]:
#Create the LLM (Shared by All Agents)

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [20]:
#REAL-WORLD TOOLS (SIMULATED FLIGHT APIs)

def fetch_flights(source, destination, date):
    return [
        {"flight": "IndiGo 6E-203", "price": 5200, "time": "06:00"},
        {"flight": "Air India AI-441", "price": 6100, "time": "09:30"},
        {"flight": "Vistara UK-835", "price": 5800, "time": "14:00"}
    ]


In [21]:
#Tool 2: Book flight (REAL ACTION)

def book_flight(flight_details):
    booking = f"""
Flight Booked Successfully ✈️
Flight: {flight_details['flight']}
Price: ₹{flight_details['price']}
Departure Time: {flight_details['time']}
"""
    with open("flight_booking.txt", "w", encoding="utf-8") as f:
        f.write(booking)
    return booking


In [28]:
#Planner Agent

def planner_agent(task):
    messages = [
        SystemMessage(content="You are a travel planner. Break the task into steps."),
        HumanMessage(content=task)
    ]
    return llm.invoke(messages).content


In [29]:
#Researcher Agent

def researcher_agent(plan):
    messages = [
        SystemMessage(content="You are a travel researcher. Identify required flight data."),
        HumanMessage(content=plan)
    ]
    return llm.invoke(messages).content


In [30]:
#Executor Agent

def executor_agent():
    flights = fetch_flights("Chennai", "Delhi", "Tomorrow")

    cheapest = min(flights, key=lambda x: x["price"])

    booking_confirmation = book_flight(cheapest)
    return booking_confirmation


In [31]:
#Critic Agent

def critic_agent(booking_output):
    messages = [
        SystemMessage(content="You are a critic. Verify booking details and clarity."),
        HumanMessage(content=booking_output)
    ]
    return llm.invoke(messages).content


In [32]:
#Orchestrator (Multi-Agent Controller)

def travel_booking_agent(user_query):
    print("🧠 Planner Agent\n")
    plan = planner_agent(user_query)
    print(plan)

    print("\n🔍 Researcher Agent\n")
    research = researcher_agent(plan)
    print(research)

    print("\n⚙️ Executor Agent (Booking Flight)\n")
    booking = executor_agent()
    print(booking)

    print("\n🧐 Critic Agent\n")
    final = critic_agent(booking)
    print(final)

    return final


In [33]:
travel_booking_agent(
    "Book the cheapest flight from Chennai to Delhi tomorrow"
)


🧠 Planner Agent

To book the cheapest flight from Chennai to Delhi tomorrow, I'll follow these steps:

**Step 1: Research and Compare Prices**

* I'll use flight comparison websites such as Skyscanner, Google Flights, or Kayak to find the cheapest flights from Chennai to Delhi.
* I'll also check the airlines' official websites, such as Air India, IndiGo, or SpiceJet, to see if they have any promotions or discounts.

**Step 2: Check Flight Availability**

* I'll check the flight schedules for tomorrow and see which flights are available.
* I'll also check for any layovers or connections, as these can sometimes be cheaper than non-stop flights.

**Step 3: Filter Results**

* I'll filter the results to show only the cheapest flights, and then sort them by price.
* I'll also consider factors such as flight duration, departure and arrival times, and airline reputation.

**Step 4: Book the Flight**

* Once I've found the cheapest flight, I'll book it online using the airline's website or a f

"To verify the booking details, I would like to clarify a few points:\n\n1. **Flight Number and Airline**: The flight number is 6E-203, which is operated by IndiGo. This seems correct.\n2. **Price**: The price of the ticket is ₹5200. I would like to confirm if this includes any additional fees, taxes, or charges.\n3. **Departure Time**: The departure time is 06:00. I would like to confirm if this is the local time at the departure airport or the scheduled time in the destination's local time.\n4. **Flight Details**: I would like to confirm the flight's origin, destination, and any layovers or connections.\n\nTo ensure clarity, could you please provide the following information:\n\n- Origin airport code\n- Destination airport code\n- Any layovers or connections\n- Check-in and baggage policies\n- Cancellation and refund policies\n\nThis will help me verify the booking details and provide a more accurate assessment."